# ResST Tutorial
Author: Jinjin Huang


## Outline
1. Installation
2. import modules
3. Reading in data
4. Enhancing gene expression matrix
5. Clustering
6. 

## 1. Installation
To install ResST package you must make sure that your python version is over 3.7.=. If you don’t know the version of python you can check it by:

In [2]:
import platform
platform.python_version()

'3.7.0'

Note: Because ResST pends on pytorch, you should make sure torch is correctly installed.

Download the package from Github and install it locally:

In [2]:
git clone https://github.com/StickTaTa/ResST_main.git
cd ResST_main
pip install -r requirements.txt